<a href="https://colab.research.google.com/github/ThisIsFarhan/FlanT5-FineTuning/blob/main/T5_FineTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%bash
pip install nltk
pip install datasets
pip install transformers[torch]
pip install tokenizers
pip install evaluate
pip install rouge_score
pip install sentencepiece
pip install huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 52.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 848.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import nltk
import evaluate
import numpy as np
from datasets import load_dataset
from transformers import T5Tokenizer, DataCollatorForSeq2Seq
from transformers import T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [3]:
# Load the tokenizer, model, and data collator
MODEL_NAME = "google/flan-t5-base"

tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [7]:
dataset = load_dataset("json",data_files="scene_descriptions_dataset.json")

Generating train split: 0 examples [00:00, ? examples/s]

In [8]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'input', 'output'],
        num_rows: 1000
    })
})

In [9]:
splitted_set = dataset["train"].train_test_split(test_size=0.2)

In [10]:
splitted_set

DatasetDict({
    train: Dataset({
        features: ['id', 'input', 'output'],
        num_rows: 800
    })
    test: Dataset({
        features: ['id', 'input', 'output'],
        num_rows: 200
    })
})

In [11]:
prefix = "Please describe the scenario based on detected objects: "
inputs = [prefix + doc for doc in splitted_set["train"]["input"]]
model_inputs = tokenizer(inputs, max_length=128, truncation=True)

In [14]:
inputs

['Please describe the scenario based on detected objects: right_input: computer, lamp, fan',
 'Please describe the scenario based on detected objects: left_input: laptop\nright_input: laptop\nbottom_input: shelf, sofa',
 'Please describe the scenario based on detected objects: right_input: refrigerator\nup_input: keyboard\nbottom_input: shelf',
 'Please describe the scenario based on detected objects: left_input: chair\nright_input: student, bottle, shelf\nbottom_input: fan, bag',
 'Please describe the scenario based on detected objects: left_input: worker, pen, printer\nright_input: bottle, drawer\nup_input: chair, microwave',
 'Please describe the scenario based on detected objects: left_input: phone, drawer, bottle\nright_input: chair, cup, phone\nbottom_input: glasses',
 'Please describe the scenario based on detected objects: left_input: woman, lamp, computer\nup_input: book, clock, monitor\nbottom_input: sofa',
 'Please describe the scenario based on detected objects: up_input: c

In [15]:
model_inputs

{'input_ids': [[863, 5530, 8, 8616, 3, 390, 30, 14619, 4820, 10, 269, 834, 77, 2562, 10, 1218, 6, 7767, 6, 1819, 1], [863, 5530, 8, 8616, 3, 390, 30, 14619, 4820, 10, 646, 834, 77, 2562, 10, 4544, 269, 834, 77, 2562, 10, 4544, 2007, 834, 77, 2562, 10, 8625, 6, 5650, 1], [863, 5530, 8, 8616, 3, 390, 30, 14619, 4820, 10, 269, 834, 77, 2562, 10, 11042, 95, 834, 77, 2562, 10, 8680, 2007, 834, 77, 2562, 10, 8625, 1], [863, 5530, 8, 8616, 3, 390, 30, 14619, 4820, 10, 646, 834, 77, 2562, 10, 3533, 269, 834, 77, 2562, 10, 1236, 6, 4782, 6, 8625, 2007, 834, 77, 2562, 10, 1819, 6, 2182, 1], [863, 5530, 8, 8616, 3, 390, 30, 14619, 4820, 10, 646, 834, 77, 2562, 10, 10416, 6, 4550, 6, 6454, 269, 834, 77, 2562, 10, 4782, 6, 10326, 95, 834, 77, 2562, 10, 3533, 6, 13333, 1], [863, 5530, 8, 8616, 3, 390, 30, 14619, 4820, 10, 646, 834, 77, 2562, 10, 951, 6, 10326, 6, 4782, 269, 834, 77, 2562, 10, 3533, 6, 4119, 6, 951, 2007, 834, 77, 2562, 10, 9832, 1], [863, 5530, 8, 8616, 3, 390, 30, 14619, 4820, 10, 

In [16]:
labels = tokenizer(text_target=splitted_set["train"]["output"],
                      max_length=512,
                      truncation=True)

In [17]:
model_inputs["labels"] = labels["input_ids"]

In [18]:
model_inputs

{'input_ids': [[863, 5530, 8, 8616, 3, 390, 30, 14619, 4820, 10, 269, 834, 77, 2562, 10, 1218, 6, 7767, 6, 1819, 1], [863, 5530, 8, 8616, 3, 390, 30, 14619, 4820, 10, 646, 834, 77, 2562, 10, 4544, 269, 834, 77, 2562, 10, 4544, 2007, 834, 77, 2562, 10, 8625, 6, 5650, 1], [863, 5530, 8, 8616, 3, 390, 30, 14619, 4820, 10, 269, 834, 77, 2562, 10, 11042, 95, 834, 77, 2562, 10, 8680, 2007, 834, 77, 2562, 10, 8625, 1], [863, 5530, 8, 8616, 3, 390, 30, 14619, 4820, 10, 646, 834, 77, 2562, 10, 3533, 269, 834, 77, 2562, 10, 1236, 6, 4782, 6, 8625, 2007, 834, 77, 2562, 10, 1819, 6, 2182, 1], [863, 5530, 8, 8616, 3, 390, 30, 14619, 4820, 10, 646, 834, 77, 2562, 10, 10416, 6, 4550, 6, 6454, 269, 834, 77, 2562, 10, 4782, 6, 10326, 95, 834, 77, 2562, 10, 3533, 6, 13333, 1], [863, 5530, 8, 8616, 3, 390, 30, 14619, 4820, 10, 646, 834, 77, 2562, 10, 951, 6, 10326, 6, 4782, 269, 834, 77, 2562, 10, 3533, 6, 4119, 6, 951, 2007, 834, 77, 2562, 10, 9832, 1], [863, 5530, 8, 8616, 3, 390, 30, 14619, 4820, 10, 

In [19]:
# We prefix our tasks with "answer the question"
prefix = "Please describe the scenario based on detected objects: "

# Define the preprocessing function

def preprocess_function(examples):
   """Add prefix to the sentences, tokenize the text, and set the labels"""
   # The "inputs" are the tokenized answer:
   inputs = [prefix + doc for doc in examples["input"]]
   model_inputs = tokenizer(inputs, max_length=128, truncation=True)

   # The "labels" are the tokenized outputs:
   labels = tokenizer(text_target=examples["output"],
                      max_length=512,
                      truncation=True)

   model_inputs["labels"] = labels["input_ids"]
   return model_inputs

In [20]:
# Map the preprocessing function across our dataset
tokenized_dataset = splitted_set.map(preprocess_function, batched=True)

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [21]:
tokenized_dataset["train"][2]

{'id': 246,
 'input': 'right_input: refrigerator\nup_input: keyboard\nbottom_input: shelf',
 'output': 'In the right side area, there was a refrigerator. a keyboard was observed top. a shelf was observed lower.',
 'input_ids': [863,
  5530,
  8,
  8616,
  3,
  390,
  30,
  14619,
  4820,
  10,
  269,
  834,
  77,
  2562,
  10,
  11042,
  95,
  834,
  77,
  2562,
  10,
  8680,
  2007,
  834,
  77,
  2562,
  10,
  8625,
  1],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'labels': [86,
  8,
  269,
  596,
  616,
  6,
  132,
  47,
  3,
  9,
  11042,
  5,
  3,
  9,
  8680,
  47,
  6970,
  420,
  5,
  3,
  9,
  8625,
  47,
  6970,
  1364,
  5,
  1]}

In [22]:
nltk.download("punkt", quiet=True)
nltk.download('punkt_tab')
metric = evaluate.load("rouge")

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [23]:
def compute_metrics(eval_preds):
   preds, labels = eval_preds

   # decode preds and labels
   labels = np.where(labels != -100, labels, tokenizer.pad_token_id)

   # Force the tokenizer to handle unknown tokens
   decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True, errors='ignore') # or errors='replace'
   decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True, errors='ignore') # or errors='replace'

   # rougeLSum expects newline after each sentence
   decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
   decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

   result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

   return result

In [24]:
# Global Parameters
L_RATE = 3e-4
BATCH_SIZE = 8
PER_DEVICE_EVAL_BATCH = 4
WEIGHT_DECAY = 0.01
SAVE_TOTAL_LIM = 3
NUM_EPOCHS = 3

# Set up training arguments
training_args = Seq2SeqTrainingArguments(
   output_dir="./results",
   evaluation_strategy="epoch",
   learning_rate=L_RATE,
   per_device_train_batch_size=BATCH_SIZE,
   per_device_eval_batch_size=PER_DEVICE_EVAL_BATCH,
   weight_decay=WEIGHT_DECAY,
   save_total_limit=SAVE_TOTAL_LIM,
   num_train_epochs=NUM_EPOCHS,
   predict_with_generate=True,
   push_to_hub=False
)

trainer = Seq2SeqTrainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_dataset["train"],
   eval_dataset=tokenized_dataset["test"],
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics
)

trainer.train()

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-24-9081caaebb41>:23: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: farhanak128 (farhanak128-comsats-university-islamabad) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,No log,0.200122,0.584655,0.432699,0.495686,0.525969


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,No log,0.200122,0.584655,0.432699,0.495686,0.525969
2,No log,0.195567,0.582287,0.420497,0.498978,0.520331
3,No log,0.186197,0.576769,0.430652,0.498948,0.517143


TrainOutput(global_step=300, training_loss=0.24381701151529947, metrics={'train_runtime': 5888.5624, 'train_samples_per_second': 0.408, 'train_steps_per_second': 0.051, 'total_flos': 143970233352192.0, 'train_loss': 0.24381701151529947, 'epoch': 3.0})

In [33]:
last_checkpoint = "./results/checkpoint-300"

finetuned_model = T5ForConditionalGeneration.from_pretrained(last_checkpoint)
tokenizer = T5Tokenizer.from_pretrained(last_checkpoint)

In [35]:
my_question = "left_input: lamp, microwave\nright_input: clock\nup_input: bottle, shelf, microwave\nbottom_input: printer"
inputs = "Please describe the scenario based on detected objects: " + my_question
input_dict = tokenizer(inputs, return_tensors="pt")  # Create a dictionary containing the input_ids
outputs = finetuned_model.generate(**input_dict, max_new_tokens=500)  # Pass the dictionary to generate
answer = tokenizer.decode(outputs[0])
from textwrap import fill

print(fill(answer, width=80))

<pad> a lamp and a microwave were detected in the left section. a clock was
observed right side. a bottle, a shelf, and a microwave were detected in the
upper section. a printer was detected in the bottom section.</s>


In [29]:
finetuned_model.save_pretrained("my_finetuned_model")  # Save the model
tokenizer.save_pretrained("my_finetuned_model")  # Save the tokenizer

('my_finetuned_model/tokenizer_config.json',
 'my_finetuned_model/special_tokens_map.json',
 'my_finetuned_model/spiece.model',
 'my_finetuned_model/added_tokens.json')

In [30]:
finetuned_model = T5ForConditionalGeneration.from_pretrained("my_finetuned_model")
tokenizer = T5Tokenizer.from_pretrained("my_finetuned_model")

In [32]:
my_question = "left_input: lamp, microwave\nright_input: clock\nup_input: bottle, shelf, microwave\nbottom_input: printer"
inputs = "Please describe the scenario based on detected objects: " + my_question
input_dict = tokenizer(inputs, return_tensors="pt")
outputs = finetuned_model.generate(**input_dict, max_new_tokens=500)
answer = tokenizer.decode(outputs[0])
print(answer)

<pad> a lamp and a microwave were detected in the left section. a clock was observed right side. a bottle, a shelf, and a microwave were detected in the upper section. a printer was detected in the bottom section.</s>


In [36]:
!zip -r /content/my_T5_finetuned_model.zip /content/my_finetuned_model

  adding: content/my_finetuned_model/ (stored 0%)
  adding: content/my_finetuned_model/generation_config.json (deflated 29%)
  adding: content/my_finetuned_model/spiece.model (deflated 48%)
  adding: content/my_finetuned_model/special_tokens_map.json (deflated 85%)
  adding: content/my_finetuned_model/config.json (deflated 62%)
  adding: content/my_finetuned_model/model.safetensors (deflated 7%)
  adding: content/my_finetuned_model/tokenizer_config.json (deflated 94%)
  adding: content/my_finetuned_model/added_tokens.json (deflated 83%)


In [38]:
from google.colab import files

files.download('/content/my_T5_finetuned_model.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>